# Example run of CL1354

**Setup Instructions**

Make sure that you have set the system variable \$EDISCS to the location of the EDiSCS directory created by Tyler (e.g., /Volumes/BAHAMUT/ediscs) which contains all of the catalogs, images, reference files, etc. The system variable \$EAZYINSTALL should be set to the location of the EAZY installation on your machine (this allows the code to correctly link the EAZY template library).

You will need to copy the following to the current working directory:

1.) FITS images (located in \$EDISCS/files/images/cl????)

2.) SExtractor files (e.g., convolution kernels)

3.) All additional SExtractor configuration files (located in \$EDISCS/files/analysis/SExtractor/)

4.) FORS photometry (located in \$EDISCS/files/analysis/fors/cl????_fors.dat)

Create a two-column file (in this example named "imglist") wherein the first column is ?img where ? is the filter letter (e.g, bimg for the B-band or rbimg for the R-band smoothed to the seeing of the B-band image) and the second column is the corresponding name of the FITS file. For CL1354, this file should contain the following: 

#contents of imglist file for CL1354
```
bimg	cl1354_b_td.fits
iimg	cl1354_i_td.fits
kimg	cl1354_k.fits
rimg	cl1354_r_td.fits
vimg	cl1354_v_td.fits
zimg	cl1354_z_td.fits
rkimg	cl1354_r_rk.fits
```

**Performing the Photometry**

Run the SExtractor batch script creation algorithm in ediscs_catalog.py to create a file that will be sourced to do all of the photometry.

In [ ]:
from ediscs import ediscs_catalog as ec

ec.photscript('imglist','cl1354')

This will create a file called "photscript" that contains the following:

#contents of photscript file for CL1354
```
sex -c ediscs.sex -BACKPHOTO_TYPE "GLOBAL" -CATALOG_NAME cl1354_r.cat -CHECKIMAGE_TYPE "-BACKGROUND,SEGMENTATION" -CHECKIMAGE_NAME "cl1354_r_td_bkgsub.fits,cl1354_r_td_segmap.fits" cl1354_r_td.fits,cl1354_r_td.fits

sex -c ediscs.sex -BACKPHOTO_TYPE "GLOBAL" -CATALOG_NAME cl1354_b.cat -CHECKIMAGE_TYPE "-BACKGROUND" -CHECKIMAGE_NAME cl1354_b_td_bkgsub.fits cl1354_r_td.fits,cl1354_b_td.fits

sex -c ediscs.sex -BACKPHOTO_TYPE "GLOBAL" -CATALOG_NAME cl1354_v.cat -CHECKIMAGE_TYPE "-BACKGROUND" -CHECKIMAGE_NAME cl1354_v_td_bkgsub.fits cl1354_r_td.fits,cl1354_v_td.fits

sex -c ediscs.sex -BACKPHOTO_TYPE "GLOBAL" -CATALOG_NAME cl1354_i.cat -CHECKIMAGE_TYPE "-BACKGROUND" -CHECKIMAGE_NAME cl1354_i_td_bkgsub.fits cl1354_r_td.fits,cl1354_i_td.fits

sex -c ediscs.sex -BACKPHOTO_TYPE "GLOBAL" -CATALOG_NAME cl1354_z.cat -CHECKIMAGE_TYPE "-BACKGROUND" -CHECKIMAGE_NAME cl1354_z_td_bkgsub.fits cl1354_r_td.fits,cl1354_z_td.fits

sex -c ediscs.sex -BACKPHOTO_TYPE "GLOBAL" -CATALOG_NAME cl1354_k.cat -CHECKIMAGE_TYPE "-BACKGROUND" -CHECKIMAGE_NAME cl1354_k_bkgsub.fits cl1354_r_td.fits,cl1354_k.fits

sex -c ediscs.sex -BACKPHOTO_TYPE "GLOBAL" -CATALOG_NAME cl1354_rk.cat -CHECKIMAGE_TYPE "-BACKGROUND" -CHECKIMAGE_NAME cl1354_r_rk_bkgsub.fits cl1354_r_td.fits,cl1354_r_rk.fits
```

**Version 6.2 Catalog Creation**

Once this file has been sourced, a SExtractor catalog for each filer will be created (e.g., "cl1354_b.cat"). You are now ready to create the version 6.2 catalog. To do so, run the following command (commands for each cluster can be found in $EDISCS/files/images/commands):

In [ ]:
from ediscs import ediscs_catalog as ec

ec.main(b='cl1354_b.cat',v='cl1354_v.cat',r='cl1354_r.cat',i='cl1354_i.cat',z='cl1354_z.cat',
        k='cl1354_k.cat',rk='cl1354_rk.cat',imglist='imglist',clname='cl1354',
        rsegmap='cl1354_r_td_segmap.fits',kpno=False,zpb=24.99,zpz=24.44,zpk=21.8897,
        idname='1354-12',errborder=1000.0,xmin=1700.0,ymax=9350.0,wkimg='cl1354_k_expmap.fits',
        fors='cl1354_fors.dat')

Most of the inputs are self-explanatory, however the parameter errborder adds a buffer zone to the edge of the image where empty apertures for the uncertainty analysis will not be placed (to ensure that noisy edges do not artificially inflate the errors). Additionally, the parameters xmin, xmax, ymin, and ymax allow the user to exclude regions of the image from the final catalog. This became necessary do to poor calibration on the edges of some images.

**Creating Quality Assessment Plots**

To check the quality of the photometry and the calibration of the WFI photometry onto the FORS filter system, a series of plots can be created. The following code will create histograms of magnitudes, plots of S/N vs. magnitude, and FORS-WFI vs. FORS magnitudes for each filter where data are available.

In [ ]:
import plot_ediscs as pe

pe.main('cl1354_catalog.fits','cl1354',forsfile='cl1354_fors.dat',classStar=0.9)

In this example, we've chosen to use stars in the WFI catalog with the SExtractor class_Star value in the R-band >= 0.9. Classifications of sources in the FORS photometry comes from an earlier version of the FORS catalog and a description can be found on the EDiSCS website.

**EAZY Photometric Redshifts**

Once the catalog has been created, copy it to another directory (e.g., a subdirectory named eazy/) and run the shell script $EDISCS/files/analysis/EAZY/eazysetup.sh to configure the directory. The file "zphot.param" will need to be edited for each cluster! Open "zphot.param" in a text editor and change the following (keywords followed by new value):

    CATALOG_FILE   randomFirst.dat
    N_MIN_COLORS   6

The N_MIN_COLORS keyword is the number of filters available for each cluster. As CL1354 has BVRIzK photometry, this value is set to 6 (which is the default in the file copied by eazysetup.sh).

To create the input ASCII catalogs for EAZY, we use two tools. As we want to test the photometric redshifts on a randomly selected sample of half of the galaxies, we use a program to generate three FITS catalogs from the version 6.2 catalog: a randomly selected half of the galaxies, the remaining half of the galaxies not selected, and all of the galaxies in one catalog. This is achieved using the following:

In [ ]:
from ediscs import ediscs_catalog as ec

ec.randomSample('cl1354_catalog.fits','BVRIzK')

The files will be named "randomFirst.fits", "randomSecond.fits", and "galaxies.fits". We then use another code to generate ASCII versions in a format that is compatible with EAZY.

In [ ]:
from ediscs import eazy as e

e.prep('randomFirst.fits','randomFirst.dat',['cB','V','R','I','z','K'],maglim=23.)

This takes the FITS catalog "randomFirst.fits" and turns it into an ASCII file called "randomFirst.dat". The list of filter names is important because it defines which filters should be selected for input into EAZY and sets up the translation file to convert between the EDisCS catalog column headers (e.g., fR3 for the R-band flux in r=3" apertures) to a format EAZY understands (the R-band flux would be labeled as F4 meaning the flux corresponding to the fourth filter in the filter definition file supplied to EAZY). Note that the B-band is called either "cB" for the CTIO MOSAIC B-band or "kB" for the KPNO MOSAIC B-band as the transmission curves of the instruments are not the same and need to be treated seperately by the code.

Next, run EAZY in the current working directory. This will generate some screen output and save binary files to the OUTPUT/ subdirectory. To view how well the EAZY redshifts compare to those from the LDP, run the following:

In [ ]:
from ediscs import eazy as e

e.plotphotspec('randomFirst.fits')

This will create two plots: "zspec_zphot.pdf" which is the EAZY photometry redshift vs. the LDP redshiftin the range $0 \leq z \leq 1$; and "photoz_residuals.pdf" which is the difference in LDP and EAZY redshifts vs. the LDP redshifts.  Another plot of the model flux residuals can be generated using another code:

In [ ]:
from threedhst import eazyPy_mod as e

e.show_fit_residuals(root='photz',FITScat='randomFirst.fits',filter='BVRIzK')

This code is a modified version of the code writted by Gabe Brammer in his ```threedhst``` package on Github. For file dependencies, the file eazyPy_mod.py must be located in your distribution of ```threedhst``` if you have not already copied it there.

**Iterating Over Zeropoint Offsets with EAZY**

EAZY does not allow for iterating over zeropoint offsets natively, therefor we must do it ourselves. First, be sure that the file "zphot.param" is configured the way you want for your cluster (i.e., make sure the correct ASCII catalog is given as input ("randomFirst.dat") and you have the correct number of filters selected). Also change the following parameter in "zphot.param":

    GET_ZP_OFFSETS   y

You will also need to create two new subdirectories called logs/ and plots/ (the plots/ directory is only necessary if you want to save the EAZY and LDP redshift comparison plots from each iteration. NOTE: these files are small in size, but there will be many of them and the plots/ directory can easily grown to be 3+ GB in size. If you do not want to save all of these plots, this can be done below.

Now run the code that will generate a grid of zeropoint offsets in V, R, and I. Note that this process will take an extremely long time (~5$-$8 seconds per grid element, e.g., ~20 hours for offsets in the range $-$0.2 to +0.2 in steps of 0.02 mag), therefor it should be run in the background or on a separate screen. 

In [ ]:
from ediscs import eazyAdjust as ea

ea.main('randomFirst.fits',maxoff=0.2,stepSize=0.02)

The program will run through each combination of V, R, and I zeropoint offsets by running EAZY, comparing the photometric redshifts and LDP values using the metric

x = $\frac{\sqrt{(z_{LDP}-z_{phot})^2}}{N}$,

and estimating the scatter in the residuals. The scatter and value of the metric above ($x$) will be saved in a file summarizing each iteration called "zpAdjust.ref" in the current directory.

NOTE: If you do not want to save the plots from each step of the iteration to conserve space, use the parametry plots=False in the code above.

**Creating Version 7.0 Catalogs**

The version 7.0 catalogs contain the redshifts from EAZY as well as updated photometry from the iterative zeropoint offset code. Before we can create this catalog, we first need a photometric redshift from EAZY for every source in the previous catalog. Thus, we need to generate another ASCII catalog:

In [ ]:
from ediscs import eazy as e

e.prep('cl1354_catalog.fits','full.dat',['cB','V','R','I','z','K'])

Make sure to edit the "zphot.param" file to use the "full.dat" catalog as input (the CATLOG_FILE keyword). Also be sure to use the best offsets in the "zphot.zeropoint" file. This file is set up in two columns of the flux identifier (e.g., F3, F4, and F5 are V, R, and I fluxes) and the multiplicative offset (i.e., 1.0 is no change to the input catalog flux). Now, run EAZY and then use the following command to generate the version 7.0 catalog:

In [ ]:
from ediscs import ediscs_catalog as ec

ec.update70('cl1354_catalog.fits','cl1354','cl1354_catalog_v7.0.fits')

This will automatically look for the best zeropoint offsets in the file "wfi_zeropoints.dat" located in \$EDISCS/files/images/ and update the photometry accordingly. If you want to supply your own offsets, set best=False and use the zpvoff, zproff, and zpioff parameters to manually input the offsets for the catalog creation.